In [ ]:
# Importamos las bibliotecas necesarias
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import LSTM, Dense
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA

# Cargamos el conjunto de datos
data = pd.read_csv("/mnt/data/datosFinales.csv")
date_range = pd.date_range(start="2019-01-01", end="2022-01-02", freq='D')
data['Date'] = date_range
data.set_index('Date', inplace=True)

# Definimos las fechas y eventos mencionados
seasons = {
    1: 'Invierno',
    2: 'Invierno',
    3: 'Invierno',
    4: 'Primavera',
    5: 'Primavera',
    6: 'Verano',
    7: 'Verano',
    8: 'Verano',
    9: 'Verano',
    10: 'Otoño',
    11: 'Otoño',
    12: 'Invierno'
}

national_holidays = ["01-jan-2019", "06-jan-2019", "01-may-2019", "14-jul-2019", "03-oct-2019",
                     "01-jan-2020", "06-jan-2020", "01-may-2020", "14-jul-2020", "03-oct-2020",
                     "01-jan-2021", "06-jan-2021", "01-may-2021", "14-jul-2021", "03-oct-2021"]

shaaban_dates = {
    2019: ("06-apr-2019", "05-may-2019"),
    2020: ("25-mar-2020", "23-apr-2020"),
    2021: ("14-mar-2021", "12-apr-2021")
}

ramadan_dates = {
    2019: ("06-may-2019", "04-jun-2019"),
    2020: ("24-apr-2020", "23-may-2020"),
    2021: ("13-apr-2021", "12-may-2021")
}

# Ahora, procederemos a realizar el análisis y entrenamiento paso a paso.
# Empezaremos por el estudio por estaciones.

# Asignamos la columna 'Season' basándonos en el mes
data['Season'] = data.index.month.map(lambda month: seasons[month])

# Definimos el conjunto de entrenamiento y validación
train_data = data.loc["2019-01-01":"2021-01-01"]
validation_data = data.loc["2021-01-02":]

# Preparamos los datos para el modelo
encoder = OneHotEncoder(sparse=False)
y_encoded = encoder.fit_transform(train_data['Season'].values.reshape(-1, 1))

# Creamos la función para generar el conjunto de datos con pasos de tiempo para LSTM
def create_multivariate_dataset(X, Y, look_back=1):
    dataX, dataY = [], []
    for i in range(len(X) - look_back):
        dataX.append(X[i:(i + look_back)])
        dataY.append(Y[i + look_back])
    return np.array(dataX), np.array(dataY)

# Vamos a definir el modelo LSTM y entrenarlo para predecir las estaciones.
# Dado que el código es extenso, primero definiremos y entrenaremos el modelo para predecir las estaciones.
# En los siguientes pasos, abordaremos los otros puntos que mencionaste.
# Nota: Este código es solo una plantilla y se espera que lo ejecutes en tu máquina local.

look_back = 30  # Por ahora, usaremos un look back de 30 días como ejemplo.
X_data, Y_data = create_multivariate_dataset(train_data[['Produccion']].values, y_encoded, look_back)

model = Sequential()
model.add(LSTM(100, input_shape=(look_back, X_data.shape[2]), return_sequences=True))
model.add(LSTM(50))
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Aquí es donde entrenarías el modelo en tu máquina local
model.fit(X_data, Y_data, epochs=50, batch_size=32, verbose=1)
